# Урок 9. Языковое моделирование

## Домашнее задание.

Разобраться с моделькой генерации текста, собрать самим или взять датасет с вебинара и обучить генератор текстов

In [1]:
import tensorflow as tf

import numpy as np
import os
import time

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
!ls

Mounted at /content/gdrive
gdrive	sample_data


In [ ]:
%cd gdrive/My\ Drive/work/GB_NLP/6

[Errno 2] No such file or directory: 'gdrive/My Drive/work/GB_NLP/6'
/content/gdrive/My Drive/work/GB_NLP/6


In [3]:
!ls

gdrive	Oruell_1984.txt  sample_data


In [4]:
path_to_file = 'Oruell_1984.txt'

In [5]:
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

Length of text: 496793 characters


In [6]:
print(text[:500])

Annotation


Своеобразный антипод второй великой антиутопии XX века – «О дивный новый мир» Олдоса Хаксли. Что, в сущности, страшнее: доведенное до абсурда «общество потребления» – или доведенное до абсолюта «общество идеи»?

По Оруэллу, нет и не может быть ничего ужаснее тотальной несвободы…

Роман публикуется в новом переводе.





* * *



Джордж Оруэлл

Часть перваяI

II

III

IV

V

VI

VII

VIII





Часть втораяI

II

III

IV

V

VI

VII



In [7]:
text = text + text

In [8]:
# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

118 unique characters


In [9]:
# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [10]:
text_as_int, text, len(text_as_int), len(text)

(array([25, 43, 43, ...,  0,  1,  0]),
 'Annotation\r\n\r\n\r\nСвоеобразный антипод второй великой антиутопии XX века – «О дивный новый мир» Олдоса Хаксли. Что, в сущности, страшнее: доведенное до абсурда «общество потребления» – или доведенное до абсолюта «общество идеи»?\r\n\r\nПо Оруэллу, нет и не может быть ничего ужаснее тотальной несвободы…\r\n\r\nРоман публикуется в новом переводе.\r\n\r\n\r\n\r\n\r\n\r\n* * *\r\n\r\n\r\n\r\nДжордж Оруэлл\r\n\r\nЧасть перваяI\r\n\r\nII\r\n\r\nIII\r\n\r\nIV\r\n\r\nV\r\n\r\nVI\r\n\r\nVII\r\n\r\nVIII\r\n\r\n\r\n\r\n\r\n\r\nЧасть втораяI\r\n\r\nII\r\n\r\nIII\r\n\r\nIV\r\n\r\nV\r\n\r\nVI\r\n\r\nVII\r\n\r\nVIII\r\n\r\nIX\r\n\r\n\r\n\r\n\r\n\r\nЧасть третьяI\r\n\r\nII\r\n\r\nIII\r\n\r\nIV\r\n\r\nV\r\n\r\nVI\r\n\r\n\r\n\r\n\r\n\r\nПриложение\r\n\r\n\r\n\r\n\r\n\r\nnotes1\r\n\r\n\r\n\r\n\r\n\r\n* * *\r\n\r\n\r\n\r\n\r\n\r\nДжордж Оруэлл\r\n\r\n1984\r\n\r\n\r\n\r\n\r\n\r\n© Школа перевода В. Баканова, 2022\r\n\r\n© Издание на русском языке AST\r\n\r\nPubl

## Train and target

In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

A
n
n
o
t


In [12]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'Annotation\r\n\r\n\r\nСвоеобразный антипод второй великой антиутопии XX века – «О дивный новый мир» Олдоса '
'Хаксли. Что, в сущности, страшнее: доведенное до абсурда «общество потребления» – или доведенное до а'
'бсолюта «общество идеи»?\r\n\r\nПо Оруэллу, нет и не может быть ничего ужаснее тотальной несвободы…\r\n\r\nРо'
'ман публикуется в новом переводе.\r\n\r\n\r\n\r\n\r\n\r\n* * *\r\n\r\n\r\n\r\nДжордж Оруэлл\r\n\r\nЧасть перваяI\r\n\r\nII\r\n\r\nIII'
'\r\n\r\nIV\r\n\r\nV\r\n\r\nVI\r\n\r\nVII\r\n\r\nVIII\r\n\r\n\r\n\r\n\r\n\r\nЧасть втораяI\r\n\r\nII\r\n\r\nIII\r\n\r\nIV\r\n\r\nV\r\n\r\nVI\r\n\r\nVII\r\n\r\nVII'


In [13]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

Print the first example input and target values:

In [14]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'Annotation\r\n\r\n\r\nСвоеобразный антипод второй великой антиутопии XX века – «О дивный новый мир» Олдоса'
Target data: 'nnotation\r\n\r\n\r\nСвоеобразный антипод второй великой антиутопии XX века – «О дивный новый мир» Олдоса '


In [15]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

In [16]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 128

# Number of RNN units
rnn_units = 1024

In [17]:
model = tf.keras.Sequential(
    [
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ]
)

In [19]:
class RNNgenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, batch_size):
        super(RNNgenerator, self).__init__()
        
        self.emb = tf.keras.layers.Embedding(vocab_size, embedding_dim)
                                 
        self.gru1 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
        self.gru2 = tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            recurrent_initializer='glorot_uniform')
                           
        self.fc = tf.keras.layers.Dense(vocab_size)

    def call(self, x):
        emb_x = self.emb(x)
        x1 = self.gru1(emb_x)
        x = x1
        for _ in range(3):
            x = self.gru2(x)
        #x = self.gru1(x)
        x = (x + x1)/2
        return self.fc(x)

model = RNNgenerator(vocab_size, embedding_dim, BATCH_SIZE)

In [20]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
                                 
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        
        tf.keras.layers.LSTM(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [21]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim),
                                 
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),

         tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=False,
                            recurrent_initializer='glorot_uniform'),
                                   
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [22]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [23]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 118) # (batch_size, sequence_length, vocab_size)


In [24]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, None, 128)         15104     
                                                                 
 gru_4 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_5 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_6 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_3 (Dense)             (None, None, 118)         120950    
                                                                 
Total params: 16,276,342
Trainable params: 16,276,342
Non-trainable params: 0
_________________________________________________________________


In [25]:
example_batch_predictions[0]

<tf.Tensor: shape=(100, 118), dtype=float32, numpy=
array([[-1.0985007e-04,  4.0920539e-04, -3.8536021e-04, ...,
        -2.6119797e-04,  5.0925904e-05,  9.9491759e-04],
       [ 2.7729498e-04,  5.6578271e-04, -3.1316729e-04, ...,
        -7.7268569e-04,  9.1529568e-04,  1.7045666e-03],
       [-7.6610944e-05,  2.6332657e-04,  7.3599274e-04, ...,
        -1.5778567e-03,  1.7136261e-03,  1.8759778e-03],
       ...,
       [-1.8572817e-03,  9.1069902e-04,  1.7343182e-03, ...,
        -2.0997836e-03,  3.3020750e-03,  2.3321826e-03],
       [-2.6992715e-03, -2.1181727e-04,  2.5660964e-03, ...,
        -1.5038790e-03,  2.8860096e-03,  2.3928476e-03],
       [-3.1070339e-03, -6.2275189e-04,  2.9677562e-03, ...,
         4.0413422e-04,  3.2463376e-03,  2.2712548e-03]], dtype=float32)>

In [26]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [27]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'своих мотивов. Не имея представления, какой именно помощи ждать от О’Брайена, Уинстон не мог словами'

Next Char Predictions: 
 '\rБЧячAuтЧхсn[юЖПTЬНSс:юОЯВЧзМ!shpУОНУpБ;ьути9ТИфНЯп[ж.cВП=тm:©[X96фsпян:,д[\r’БН\r\nд[oРхсAБbРrм:T*Śs?'


## Train the model

In [28]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 118)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.770965


In [29]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [33]:
!mkdir training_checkpoints

In [ ]:
# !rm -rf ./training_checkpoints

In [36]:
!ls ./training_checkpoints

In [37]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_freq=88*3,
    save_weights_only=True)

### Execute the training

In [38]:
EPOCHS = 240

In [39]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/240
153/153 [==============================] - 29s 180ms/step - loss: 2.7132
Epoch 2/240
153/153 [==============================] - 30s 187ms/step - loss: 2.1157
Epoch 3/240
153/153 [==============================] - 30s 183ms/step - loss: 1.6998
Epoch 4/240
153/153 [==============================] - 31s 189ms/step - loss: 1.4440
Epoch 5/240
153/153 [==============================] - 31s 187ms/step - loss: 1.2587
Epoch 6/240
153/153 [==============================] - 31s 192ms/step - loss: 1.0863
Epoch 7/240
153/153 [==============================] - 31s 193ms/step - loss: 0.9070
Epoch 8/240
153/153 [==============================] - 32s 191ms/step - loss: 0.7238
Epoch 9/240
153/153 [==============================] - 32s 196ms/step - loss: 0.5570
Epoch 10/240
153/153 [==============================] - 31s 194ms/step - loss: 0.4147
Epoch 11/240
153/153 [==============================] - 32s 195ms/step - loss: 0.3091
Epoch 12/240
153/153 [==============================] - 31s 193

## Generate text

In [40]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_240'

In [41]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

In [42]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, None, 128)         15104     
                                                                 
 gru_7 (GRU)                 (None, None, 1024)        3545088   
                                                                 
 gru_8 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 gru_9 (GRU)                 (None, None, 1024)        6297600   
                                                                 
 dense_4 (Dense)             (None, None, 118)         120950    
                                                                 
Total params: 16,276,342
Trainable params: 16,276,342
Non-trainable params: 0
_________________________________________________________________


In [43]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [44]:
text_ = generate_text(model, start_string=u"но в тот миг его так ")
print(text_)

но в тот миг его так за пра ве ни пра в провестово но оли по О ведино ода проло не м нело нела нили в но о по Ть, стоволь стьстото на подусти преце праконало сучит прошия Пре сь ода па ово нобеесерижело ры пониля их овово пу говено поне прусо немина по м Ведаль поре, пой нин воро поми тодко ве стомии сто нолому о ндов ска не пратиели по овоне оле и и, стомиееразам неда вореро налобу ноны. доно прав пееннотоми сти н.





















































In [45]:
len(text_)

521